In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import pandasql as ps

In [2]:
df_request = pd.read_csv('~/Desktop/request.csv')
df_ride = pd.read_csv('~/Desktop/ride.csv')

# Create thresholds and then filter requests data to uncover fraudulent passengers

In [3]:
#alter objects to datetime, create a time to cancel column
df_request['cancelled_at'] = pd.to_datetime(df_request['cancelled_at'])
df_request['created_at'] = pd.to_datetime(df_request['created_at'])
df_request['time_to_cancel'] = df_request['cancelled_at'] - df_request['created_at']

In [4]:
df_request.head()

,id_request,id_passenger,id_city,from_latitude,from_longitude,to_latitude,to_longitude,created_at,cancelled_at,timedout_at,passenger_device,passenger_payment_mean,passenger_udid,distance_estimate,duration_estimate,time_to_cancel
0,91570297,7079607,1,19.531442,-99.228742,19.537476,-99.230058,2019-09-14 03:01:42,2019-09-14 03:01:44,NaN,998b8d09847b23d5/b25f5721575dc9ca,cash,508x8zax09c916c2508x8zax09c916c2508x8zax,1.20,NaN,00:00:02
1,98877504,7123610,1,19.361999,-99.002745,19.354359,-99.010834,2019-09-02 11:34:32,NaT,NaN,daisy_sprout/Mi A2 Lite,cash,4293616092704954293616092704954293616092,1.35,224.0,NaT
2,93230913,808725,1,19.502474,-99.095594,19.482185,-99.111334,2019-09-18 12:29:57,2019-09-18 12:30:41,NaN,albus/Moto Z2 Play,cash,9199666486898729199666486898729199666486,3.40,NaN,00:00:44
3,99055888,7093734,1,19.466261,-99.091820,19.447416,-99.152832,2019-09-07 00:42:41,2019-09-07 00:44:28,NaN,"iPhone7,2",cash,78908Z84B8Z1-BA38-0ZA9-51Y7-5243Y11XXBB2,9.95,NaN,00:01:47
4,92140194,8925095,2,25.782421,-100.398782,25.763432,-100.265818,2019-09-15 22:00:06,NaT,2019-09-15 22:00:30.000,j7y17lte/SM-J730GM,cash,9137916496701999137916496701999137916496,23.45,NaN,NaT


In [5]:
#exclude requests that were not cancelled, convert time to cancel to an integer
bool_series = pd.notnull(df_request['cancelled_at'])
df_time_to_cancel = df_request[bool_series]
df_time_to_cancel['time_to_cancel'] = pd.to_numeric(df_time_to_cancel['time_to_cancel'])/10**9


/Users/rolandcrouch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [6]:
df_time_to_cancel.head()

,id_request,id_passenger,id_city,from_latitude,from_longitude,to_latitude,to_longitude,created_at,cancelled_at,timedout_at,passenger_device,passenger_payment_mean,passenger_udid,distance_estimate,duration_estimate,time_to_cancel
0,91570297,7079607,1,19.531442,-99.228742,19.537476,-99.230058,2019-09-14 03:01:42,2019-09-14 03:01:44,NaN,998b8d09847b23d5/b25f5721575dc9ca,cash,508x8zax09c916c2508x8zax09c916c2508x8zax,1.20,NaN,2.0
2,93230913,808725,1,19.502474,-99.095594,19.482185,-99.111334,2019-09-18 12:29:57,2019-09-18 12:30:41,NaN,albus/Moto Z2 Play,cash,9199666486898729199666486898729199666486,3.40,NaN,44.0
3,99055888,7093734,1,19.466261,-99.091820,19.447416,-99.152832,2019-09-07 00:42:41,2019-09-07 00:44:28,NaN,"iPhone7,2",cash,78908Z84B8Z1-BA38-0ZA9-51Y7-5243Y11XXBB2,9.95,NaN,107.0
5,7236680,8801410,1,19.657555,-99.074321,19.673426,-99.087038,2019-09-30 00:03:37,2019-09-30 00:03:40,NaN,j2y18lte/SM-J250M,cash,9199873948565369199873948565369199873948,3.00,NaN,3.0
10,95864779,10766,1,19.434386,-99.085298,19.294537,-99.152945,2019-09-22 06:57:43,2019-09-22 07:00:56,NaN,a30/SM-A305G,cash,9104877606699389104877606699389104877606,21.08,NaN,193.0


In [7]:
# average time to cancel per passenger and reset index
df_time_to_cancel = df_time_to_cancel.groupby('id_passenger')['time_to_cancel'].mean()
df_time_to_cancel = df_time_to_cancel.reset_index()
df_time_to_cancel.head()

,id_passenger,time_to_cancel
0,1,82.666667
1,2,102.666667
2,3,157.125000
3,8,98.285714
4,10,151.000000


In [8]:
df_time_to_cancel.head()

,id_passenger,time_to_cancel
0,1,82.666667
1,2,102.666667
2,3,157.125000
3,8,98.285714
4,10,151.000000


In [9]:
# work out the average requests per passenger
requests_per_passenger = df_request['created_at'].count()/df_request['id_passenger'].nunique()
requests_per_passenger

12.79278067648026

In [10]:
# workout the average cancel rate per passenger
df_cancel_rate = df_request.groupby('id_passenger').agg({'created_at':'count','cancelled_at':'count'})
df_cancel_rate = df_cancel_rate.reset_index()
df_cancel_rate['cancel_rate'] = df_cancel_rate['cancelled_at']/df_cancel_rate['created_at']
cancel_rate_per_passenger = df_cancel_rate['cancel_rate'].mean()
cancel_rate_per_passenger

0.3321831804030902

In [11]:
# union cancel rate and time to cancel
# apply filters as mentioned in presentation to acquire fraudulent passengers
df_union = df_cancel_rate.merge(df_time_to_cancel, on = 'id_passenger')
df_fraudulent_passengers = df_union[(df_union.time_to_cancel <= 5) & (df_union.cancel_rate > 
                                   (cancel_rate_per_passenger + ((((cancel_rate_per_passenger*100)**(1/2))/100)*3)))&
                                   (df_union.created_at > (requests_per_passenger + 
                                    (requests_per_passenger**(1/2))*3))]

In [12]:
# sort values to get a fraudulent passenger with the most requests
df_fraudulent_passengers.sort_values(['created_at'], ascending = False ,inplace = True)
df_fraudulent_passengers.head()

/Users/rolandcrouch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id_passenger,created_at,cancelled_at,cancel_rate,time_to_cancel
159480,898071,412,244,0.592233,4.196721
545882,8962259,392,389,0.992347,3.473008
32626,88680,379,362,0.955145,2.737569
356244,7928436,361,255,0.706371,3.811765
496930,8831926,359,314,0.874652,2.592357


In [36]:
# join the fraudulent passengers onto request table and count number of requests they are responsible for
df_fraud_requests = df_request.merge(df_fraudulent_passengers, on = 'id_passenger', how = 'left')
df_fraud_requests_count = df_fraud_requests[['created_at_x','created_at_y']]
df_fraud_requests_count.count()

created_at_x    10425144
created_at_y     1444285
dtype: int64

In [31]:
# % of requests made by fraudulent passengers
1444285/10425144

0.13853861395103992

In [37]:
# number of cancellations made by fraudulent passengers
df_fraud_cancels = df_fraud_requests[['cancelled_at_x','cancelled_at_y']]
df_fraud_cancels.count()

cancelled_at_x    4154018
cancelled_at_y    1444285
dtype: int64

In [38]:
# % of cancellations made by fraudalent passengers
1444285/4154018

0.3476838569308077

In [13]:
# select the passenger with most requests, sort the data so requests are in time order, 
# send to excel for ease of reading
df_fraudulent_passenger = df_request[df_request.id_passenger == 898071]
df_fraudulent_passenger.sort_values(['created_at'], ascending = True ,inplace = True)
df_fraudulent_passenger.to_excel('~/Desktop/fraud_passenger.xlsx')

/Users/rolandcrouch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Look at rides table to match fraudulent requests

In [14]:
# convert object to datetime
# Look at rides take only between specific time points from previous step in Excel
# Again send to excel for ease of reading
df_ride['created'] = pd.to_datetime(df_ride['created'])
df_ride_time = df_ride[(df_ride['created'] > '2019-09-02  06:18:15') & 
                       (df_ride['created'] < '2019-09-02  06:50:00')]  
df_ride_time.sort_values(['created'], ascending = True ,inplace = True)
df_ride_time.to_excel('~/Desktop/fraud_ride.xlsx')

/Users/rolandcrouch/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


# % of rides with revenue of 0

In [21]:
df_ride[df_ride.revenue == 0].count()/ df_ride.count()

id_ride                0.248760
id_request             0.248760
id_driver              0.248760
distance               0.248760
revenue                0.248760
created                0.248760
ride_total_distance    0.058005
duration               0.058292
driver_udid            0.248760
started_at             0.058304
ended_at               0.058040
dtype: float64